In [1]:
# Import CANVAS_API_KEY from the .env file
from dotenv import load_dotenv
load_dotenv(verbose=True)

import os
API_KEY = os.getenv("CANVAS_API_KEY")
BASE_URL = os.getenv("CANVAS_BASE_URL")


In [2]:
import pandas as pd
import requests
import json
import numpy as np

In [3]:
def my_request(API_URL):
    headers = {'Authorization': 'Bearer '+ API_KEY}
    r = requests.get(BASE_URL+API_URL, headers=headers)
    return r.json()

In [4]:
r = my_request('/api/v1/courses?enrollment_type=teacher')

response= pd.DataFrame.from_dict(r)
course_ids = response[response['name'].str.contains('AI')]['id']


0    1116
1    1202
2    1394
3    1574
4    1682
Name: id, dtype: int64

```
id	name
1116	AI-1
1202	AI-2
1394	AI-3
1574	AI-4
1682	AI-5
```

In [5]:
#let's start by using AI-4 id:1574
# course_id =course_ids[4]

### Get student summaries

In [6]:
# #Get student_summaries
# r = my_request('/api/v1/courses/{}/analytics/student_summaries?per_page=200'.format(course_id))

# response= pd.DataFrame.from_dict(r)

In [7]:
# tardiness_summary = pd.DataFrame(list(response.tardiness_breakdown))
# student_summaries = response.merge(tardiness_summary, left_index=True, right_index=True).drop(['tardiness_breakdown'], axis='columns')
# student_summaries.head()

### Getting Student History

In [8]:
# # Get student history
# # let's look at id: 75494
# course_ids = '1574'
# student_id = '75494'
# r = my_request('/api/v1/courses/{}/analytics/users/{}/activity?per_page=200'.format(course_ids,student_id))


In [9]:
# r['page_views']

In [10]:
# pd.DataFrame(list(zip(list(r['page_views']),r['page_views'].values())),columns=['date', 'pageviews']).head()

In [11]:
# pd.DataFrame.from_dict(r['participations']).head()

### Get student outcomes

In [12]:
# #get sections
# course_id = 1574
                                      
# r = my_request('/api/v1/courses/{}/sections?per_page=200'.format(course_id))

In [13]:
# pd.DataFrame.from_dict(r)[['id','name']]

## Extract Valid assignments

In [14]:
# #get assignment id (contains all sections)
# course_id = '1574'
                                      
# r = my_request('/api/v1/courses/{}/assignments?per_page=200'.format(course_id))

In [15]:
# all_assignments = pd.DataFrame(r)
# all_assignments['name']

In [16]:
# required_assignments = all_assignments[all_assignments.name.str.contains('Assignment')
#                              ^all_assignments.name.str.contains('Capstone')]
# required_assignments[['id','name']].head()

In [17]:
# capstones = all_assignments[all_assignments.name.str.contains('Capstone')]
# capstones[['id','name']].head()

## Assignments per student

In [18]:
# #get list of assignments per student
# course_id = '1574'
# student_id = '71164'

# r = my_request('/api/v1/courses/{}/students/submissions?student_ids[]={}&per_page=200'.format(course_id,student_id))

In [20]:
# pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']]

In [21]:
# student_submissions = pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']]
# student_submissions['grade'].replace(to_replace='complete',value=1, inplace=True)
# student_submissions['grade'].fillna(0,inplace=True)
# student_submissions['grade'] = student_submissions['grade'].astype(int) #convert boolean to int
# student_submissions['late'] = student_submissions['late'].astype(int)
# student_submissions['missing'] = student_submissions['missing'].astype(int)
# student_submissions

In [22]:
def get_required_submissions_ids(course_id) : 
    '''extracts the required course ids for the course returns two dataframes:
    - required_assignments
    - capstones
    each with two columns 'id' and 'name' 
    example usage: 
        required_assignments, capstones = get_required_submissions_ids(course_id) '''
    
    r = my_request('/api/v1/courses/{}/assignments?per_page=200'.format(course_id))
    all_assignments = pd.DataFrame(r)
    required_assignments = all_assignments[all_assignments.name.str.contains('Assignment')
                             ^all_assignments.name.str.contains('Capstone')]
    capstones = all_assignments[all_assignments.name.str.contains('Capstone')]
    return required_assignments[['id','name']], capstones[['id','name']]

In [42]:
# need to get only active students!! course 1682 failing at student 72758
def get_list_of_students(course_id) : 
    '''get the list of students in a panda series.
    Excample usage:
    student_ids = get_list_of_students(course_id)'''
    
    r = my_request('/api/v1/courses/{}/enrollments?type[]=StudentEnrollment&state[]=active&per_page=200'.format(course_id))
    
    return pd.DataFrame.from_dict(r)['user_id']


In [36]:
def check_student_passing_criteria(student_id, course_id, required_assignments, capstones) :
    '''checks if the student passed or failed the course. returns boolean'''
    r = my_request('/api/v1/courses/{}/students/submissions?student_ids[]={}&per_page=200'.format(course_id,student_id))
    student_submissions = pd.DataFrame.from_dict(r)[['grade','assignment_id','late','missing','seconds_late']] #could reduce to 'assignment_id' and 'grade'
    student_submissions['grade'].replace(to_replace='complete',value=1, inplace=True)
    student_submissions['grade'].replace(to_replace='incomplete',value=0, inplace=True)
    student_submissions['grade'].fillna(0,inplace=True)
    student_submissions['grade'] = student_submissions['grade'].astype(int) #convert boolean to int
    #student_submissions['late'] = student_submissions['late'].astype(int) # not needed as of right now. could check if too late...
    #student_submissions['missing'] = student_submissions['missing'].astype(int)
    
    capstone_check = pd.merge(student_submissions, capstones, left_on='assignment_id', right_on='id', how='inner')['grade'].sum()
    if capstone_check == 1 : 
        # meets the capstone passing criteria
        # checking the next criteria
        assignment_check = pd.merge(student_submissions, required_assignments, left_on='assignment_id', right_on='id', how='inner')['grade'].sum()
        if assignment_check >= 7 :
            # meets both criterias --> pass
            return True
    else :
        # didn't meet at least one criteria
        return False

In [29]:
# calculate pass fail for entire course
def check_passing_criteria_from_course_id(course_id) :
    '''gets student pass fail for entire course'''
    # get list of required assignments
    print('Getting list of required assignments...', end=" ")
    required_assignments, capstones = get_required_submissions_ids(course_id)
    print('DONE')
    
    # get list of students
    print('Getting list of students...', end=" ")
    student_ids = get_list_of_students(course_id)
    print('DONE')
    
    print('Checking every single student...\n')
    for s_id in student_ids :
        print(s_id, end=" ")
        print(check_student_passing_criteria(s_id, course_id, required_assignments, capstones))

```
1116	AI-1
1202	AI-2
1394	AI-3
1574	AI-4
1682	AI-5
```

In [44]:
check_passing_criteria_from_course_id('1116')

Getting list of required assignments... DONE
Getting list of students... DONE
Checking every single student...

47649 False
52263 False
24036 False
52137 False
52267 False
52942 False
52140 False
52181 False
52048 False
51999 False
52802 False
52001 False
52003 False
52143 False
24907 False
52005 False
51929 False
52007 False
52009 False
52011 False
51931 True
52419 False
51934 False
52013 False
52313 False
52016 False
52018 False
51937 False
52021 False
52066 False
52023 False
51946 False
52027 False
50876 False
52029 False
52032 False
52615 False
2965 False
52036 False
52039 True
52042 False
52186 False
52000 False
44532 False
53118 False
52004 False
51940 False
52075 False
52006 False
52008 False
52010 False
44537 False
11584 False
52012 False
52079 False
52190 False
52015 False
42254 False
52017 False
52020 False
51943 False
52024 False
52026 False
52030 False
52033 False
52193 True
52197 False
52035 False
52045 False
52083 False
52041 False
52044 False
52046 False
52087 False
5220

## check pass fail for the entire course